<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

The following example plots the cross power spectral density of two signals with
some common features:


In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
rng = np.random.default_rng()

# Generate two test signals with some common features:
N, fs = 100_000, 10e3  # number of samples and sampling frequency
amp, freq = 20, 1234.0  # amplitude and frequency of utilized sine signal
noise_power = 0.001 * fs / 2
time = np.arange(N) / fs
b, a = signal.butter(2, 0.25, 'low')
x = rng.normal(scale=np.sqrt(noise_power), size=time.shape)
y = signal.lfilter(b, a, x)
x += amp*np.sin(2*np.pi*freq*time)
y += rng.normal(scale=0.1*np.sqrt(noise_power), size=time.shape)

# Compute and plot the magnitude of the cross spectral density:
nperseg, noverlap, win = 1024, 512, 'hann'
f, Pxy = signal.csd(x, y, fs, win, nperseg, noverlap)
fig0, ax0 = plt.subplots(tight_layout=True)
ax0.set_title(f"CSD ({win.title()}-window, {nperseg=}, {noverlap=})")
ax0.set(xlabel="Frequency $f$ in kHz", ylabel="CSD Magnitude in V²/Hz")
ax0.semilogy(f/1e3, np.abs(Pxy))
ax0.grid()
plt.show()

The cross spectral density is calculated by taking the average over the time slices
of a spectrogram:


In [ ]:
SFT = signal.ShortTimeFFT.from_window('hann', fs, nperseg, noverlap,
                                      scale_to='psd', fft_mode='onesided2X',
                                      phase_shift=None)
Sxy1 = SFT.spectrogram(y, x, detr='constant', k_offset=nperseg//2,
                       p0=0, p1=(N-noverlap) // SFT.hop)
Pxy1 = Sxy1.mean(axis=-1)
np.allclose(Pxy, Pxy1)  # same result as with csd()

True

As discussed in the Notes section, the results of using an approach analogous to
the code snippet above and the `csd` function may deviate due to implementation
details.

Note that the code snippet above can be easily adapted to determine other
statistical properties than the mean value.
